In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACKING_V2"]="true"

from langchain_community.llms import Ollama
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
## Loading Webpage
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://python.langchain.com/docs/introduction/")
document=loader.load()
document

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memor

In [5]:
## Spliting the document
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
document_chunks=text_splitter.split_documents(document)
document_chunks

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a 

In [6]:
## Creating embeddings and storing them in faiss vectorstore
from langchain_community.embeddings import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="gemma2:2b")
from langchain_community.vectorstores import FAISS
vectorstore=FAISS.from_documents(document_chunks,embeddings)


C:\Users\bagna\AppData\Local\Temp\ipykernel_9320\2342366952.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model="gemma2:2b")


In [7]:
##querying the vectorstore
query="LangChain simplifies every stage of the LLM application lifecycle:"
result=vectorstore.similarity_search(query)
result[0].page_content

'LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'

In [8]:
##retreival chains and document chains
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """
    Answer the following question based on the following context:
    <context>
    {context}
    </context>
    
    """
)

llm=Ollama(model="gemma2:2b")
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

C:\Users\bagna\AppData\Local\Temp\ipykernel_9320\1737880484.py:14: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="gemma2:2b")


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the following context:\n    <context>\n    {context}\n    </context>\n    \n    '), additional_kwargs={})])
| Ollama(model='gemma2:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [9]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":query,
    "context":[Document(page_content='''LangChain is a framework for developing applications powered by large language models (LLMs).

LangChain simplifies every stage of the LLM application lifecycle:

Development: Build your applications using LangChain's open-source components and third-party integrations. Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support.
Productionization: Use LangSmith to inspect, monitor and evaluate your applications, so that you can continuously optimize and deploy with confidence.
Deployment: Turn your LangGraph applications into production-ready APIs and Assistants with LangGraph Platform.''')]
})

'What is the primary purpose of LangChain? \n\n**Answer:**  LangChain simplifies the development, productionization, and deployment of LLM-powered applications. It provides a framework for building apps using LLMs, making them more accessible to developers. \n'

In [10]:
## Retreival chains
from langchain.chains.retrieval import create_retrieval_chain
reteiver=vectorstore.as_retriever()
reteiver_chain=create_retrieval_chain(reteiver,document_chain)

response=reteiver_chain.invoke({
    "input":"What are the stages of LLM application lifecycle?"
})

response["answer"]

'Let\'s dive into LangChain tutorials. It seems you are looking for information on how to develop applications using large language models (LLMs). \n\nHere\'s a breakdown of what the provided text reveals about LangChain and where you can find more details: \n\n**What is LangChain?**\n\nLangChain is like a toolbox designed specifically for building sophisticated LLM-powered applications. Think of it as LEGO bricks for AI tasks! \n\n* **Purpose:**  It simplifies complex LLM application development, from creating chatbots to advanced text analysis.\n* **Key Components:** LangChain provides pre-built "components" (tools) that you can combine and tweak to build your desired functionality: \n    * **Chat Models:** Build conversational AI (like a chatbot).\n    * **Embeddings (Vector Store):**  Store information in ways LLMs understand, enabling tasks like search and similarity analysis.\n    * **Retrievers:** Tools for accessing and working with external data sources, like databases or the 